In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
import os
import sys
current_dir = os.path.dirname(os.path.abspath(''))
sys.path.append(current_dir)
import RNN_KAN

In [24]:
# load test data
data_for_test = pd.read_csv('test_data.csv')
data_for_test['TIME'] = pd.to_datetime(data_for_test['TIME'])
data_for_test.set_index('TIME', inplace=True)

cnd_K = 4
evp_K = 6
cmp_K = 7
K = max(cnd_K, evp_K, cmp_K)

hidden_size = (30,30)
kan_hidden_size = 20
grid_size = 10

In [9]:
data_for_test.head()

,Tin,Tout,frequency,EEV,p_cnd,p_evp,h_cmp_out,h_cmp_in,h_cnd_out,h_cmp_out_p
TIME,,,,,,,,,,
2024-02-26 00:00:00,24.142857,2.657143,40,158,2664711.685,839464.3302,588743.8076,245070.2918,281977.4573,593965.3322
2024-02-26 00:01:00,24.166667,2.800000,40,160,2664711.685,839464.3302,593965.3322,515479.6690,281977.4573,598034.3997
2024-02-26 00:02:00,24.147619,2.800000,40,164,2664711.685,839464.3302,598034.3997,515479.6690,281977.4573,580692.7603
2024-02-26 00:03:00,24.161905,2.800000,40,165,2664711.685,828165.8721,580692.7603,470334.1686,281977.4573,583223.8769
2024-02-26 00:04:00,24.150000,2.833333,40,164,2664711.685,839464.3302,583223.8769,200002.0623,281977.4573,588743.8076


In [22]:
# load training data
train_data_set = np.load('training_data.npy', allow_pickle=True)
print(train_data_set.shape)
train_data_set, _  = train_test_split(train_data_set, test_size=0.8, random_state=42)

(1313, 7, 11)


In [23]:
# prepare the training data
cmp_trainset_input = train_data_set[:, :, [0,1,2,3,7,5,4]].astype(float)
cmp_trainset_output = train_data_set[:, -1, [9]].astype(float)
cnd_trainset_input = train_data_set[:, :, [0,1,2,3,6]].astype(float)
cnd_trainset_output = train_data_set[:, -1, [4,8]].astype(float)
evp_trainset_input = train_data_set[:, :, [0,1,2,3,8]].astype(float)
evp_trainset_output = train_data_set[:, -1, [5,7]].astype(float)

In [25]:
# train the RNN-KAN model
cmp_model, cmp_input_scaler, cmp_output_scaler = RNN_KAN.train(cmp_trainset_input, cmp_trainset_output, hidden_size[1], kan_hidden_size, grid_size)
cnd_model, cnd_input_scaler, cnd_output_scaler = RNN_KAN.train(cnd_trainset_input, cnd_trainset_output, hidden_size[0], kan_hidden_size, grid_size)
evp_model, evp_input_scaler, evp_output_scaler = RNN_KAN.train(evp_trainset_input, evp_trainset_output, hidden_size[0], kan_hidden_size, grid_size)

checkpoint directory created: ./model
saving model version 0.0
Epoch [10/150], Train Loss: 0.2252, validate Loss: 0.2666
Best Train Loss: 0.1259, Best validate Loss: 0.2324
checkpoint directory created: ./model
saving model version 0.0
Epoch [10/150], Train Loss: 0.2255, validate Loss: 0.1695
Epoch [20/150], Train Loss: 0.0943, validate Loss: 0.1554
Best Train Loss: 0.0860, Best validate Loss: 0.1449
checkpoint directory created: ./model
saving model version 0.0
Epoch [10/150], Train Loss: 0.3942, validate Loss: 0.3773
Epoch [20/150], Train Loss: 0.1007, validate Loss: 0.2601
Best Train Loss: 0.0665, Best validate Loss: 0.2574


In [26]:
N = data_for_test.shape[0]
P_cnd, P_evp, H_ref_cnd_out, H_ref_evp_out, H_ref_com_out = [], [], [], [], []

# assign the initial values
h_cmp_out_array = data_for_test.iloc[0:K-1,6].values
h_cnd_out_array = data_for_test.iloc[0:K-1,8].values
h_evp_out_array = data_for_test.iloc[0:K-1,7].values
p_cnd_array = data_for_test.iloc[0:K-1,4].values
p_evp_array = data_for_test.iloc[0:K-1,5].values

# for condenser as starting subsystem
h_ref_com_out = data_for_test.iloc[K-1,6]

In [27]:
# predict the outputs
for t in range(N-K):
    op_paras = data_for_test.iloc[t:t+K,0:4]
    H_ref_com_out.append(h_ref_com_out)

    # condenser subsystem
    # predict condenser pressure and the outlet enthalpy
    cnd_rnn_x = op_paras[['Tin','Tout','frequency','EEV']].values[-cnd_K:]
    h_cmp_out_array = np.append(h_cmp_out_array,h_ref_com_out)
    cnd_rnn_x = np.hstack((cnd_rnn_x, h_cmp_out_array[-cnd_K:].reshape(cnd_K,1)))
    cnd_y, cnd_y_inv = RNN_KAN.predict(cnd_model, cnd_rnn_x, cnd_input_scaler, cnd_output_scaler)
    p_cnd, h_ref_cnd_out = cnd_y_inv[0]

    h_cmp_out_array = h_cmp_out_array[1:]
    
    P_cnd.append(p_cnd)
    H_ref_cnd_out.append(h_ref_cnd_out)

    # evaporator subsystem
    # predict evaporator pressure and the outlet enthalpy
    evp_rnn_x = op_paras[['Tin','Tout','frequency','EEV']].values[-evp_K:]
    h_cnd_out_array = np.append(h_cnd_out_array, h_ref_cnd_out)
    evp_rnn_x = np.hstack((evp_rnn_x, h_cnd_out_array[-evp_K:].reshape(evp_K,1)))
    evp_y, evp_y_inv = RNN_KAN.predict(evp_model, evp_rnn_x, evp_input_scaler, evp_output_scaler)
    p_evp, h_ref_evp_out = evp_y_inv[0]
    
    P_evp.append(p_evp)
    H_ref_evp_out.append(h_ref_evp_out)
    
    h_cnd_out_array = h_cnd_out_array[1:]
    
    # compressor subsystem
    # predict compressor outlet enthalpy
    cmp_rnn_x = op_paras[['Tin','Tout','frequency','EEV']].values[-cmp_K:]
    h_evp_out_array = np.append(h_evp_out_array, h_ref_evp_out)
    p_evp_array = np.append(p_evp_array, p_evp)
    p_cnd_array = np.append(p_cnd_array, p_cnd)
    cmp_rnn_x = np.hstack((cmp_rnn_x, h_evp_out_array[-cmp_K:].reshape(cmp_K,1), p_evp_array[-cmp_K:].reshape(cmp_K,1), p_cnd_array[-cmp_K:].reshape(cmp_K,1)))
    cmp_y, cmp_y_inv = RNN_KAN.predict(cmp_model, cmp_rnn_x, cmp_input_scaler, cmp_output_scaler)
    h_ref_com_out = cmp_y_inv[0][0]
    
    h_evp_out_array = h_evp_out_array[1:]
    p_evp_array = p_evp_array[1:]
    p_cnd_array = p_cnd_array[1:]

# summarize the results
predict_data = list(zip(P_cnd, P_evp, H_ref_com_out, H_ref_evp_out, H_ref_cnd_out))
predict_df = pd.DataFrame(predict_data, index=data_for_test.index[K-1:-1], columns=['P_cnd', 'P_evp','H_cmp_out','H_evp_out','H_cnd_out'])
predict_df['delta_H_cnd'] = predict_df['H_cmp_out']-predict_df['H_cnd_out']

In [28]:
# save the results
predict_df.to_csv('prediction_results')

In [29]:
predict_df.head()

,P_cnd,P_evp,H_cmp_out,H_evp_out,H_cnd_out,delta_H_cnd
TIME,,,,,,
2024-02-26 00:06:00,2441701.00,745419.7500,594373.1239,349619.59375,275619.4375,318753.6864
2024-02-26 00:07:00,2529279.50,747866.0000,581301.2500,347989.31250,278869.1250,302432.1250
2024-02-26 00:08:00,2521424.25,744091.4375,581873.2500,336816.18750,277895.6875,303977.5625
2024-02-26 00:09:00,2515440.00,747207.6875,580959.3750,345003.03125,277558.2500,303401.1250
2024-02-26 00:10:00,2535094.25,746557.8750,581386.5625,339347.84375,278251.0000,303135.5625
